<a href="https://colab.research.google.com/github/yongsa-nut/TU_CN240_DataScience_671/blob/main/CN240_Lecture_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 6: Data Wrangling and Data Cleaning

## Overview
- Reshaping data
- Joining data
- Examples
- Dealing with Text
- Missing data (if we have time today.)

## Reshaping Data

### `melt()`
- `melt()` turn a wide table into a long table.
- The format that we want is row = one observation, column = one variable.
- Syntax: `melt(id_vars, value_vars)`
  - `id_vars`: specifies the column(s) that should **remain unchanged**
  - `value_vars`: Column(s) to be a new column.

<img src= 'https://drive.google.com/thumbnail?id=1JK2hmw6XcfajDNgQgryKLMqXoVm9QF2o&sz=w1000'></img>

In [ ]:
import pandas as pd

In [ ]:
# load the data
# Get the data from github
wide_data = pd.read_csv('/content/fertility-two-countries-example.csv')
wide_data

,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Germany,2.41,2.44,2.47,2.49,2.49,2.48,2.44,2.37,2.28,...,1.36,1.36,1.37,1.38,1.39,1.40,1.41,1.42,1.43,1.44
1,South Korea,6.16,5.99,5.79,5.57,5.36,5.16,4.99,4.85,4.73,...,1.20,1.21,1.23,1.25,1.27,1.29,1.30,1.32,1.34,1.36


In [ ]:
new_data = wide_data.melt(id_vars = ['country'],
                          value_vars = wide_data.columns[1:],
                          var_name= 'year',
                          value_name='fertility'
                          )
new_data

,country,year,fertility
0,Germany,1960,2.41
1,South Korea,1960,6.16
2,Germany,1961,2.44
3,South Korea,1961,5.99
4,Germany,1962,2.47
...,...,...,...
107,South Korea,2013,1.32
108,Germany,2014,1.43
109,South Korea,2014,1.34
110,Germany,2015,1.44


### `pivot`

- We can convert from long data to wide data using `pivot`
- Syntax: `df.pivot(index=ids, columns='var', values='val)`
  - The `index` parameter specifies the columns that should **remain as index** columns.
  -  The `columns` parameter specifies the column that will be used to **create new columns**.
  - The `values` parameter specifies the column that contains the **values to be filled** in the new columns.

<img src= 'https://drive.google.com/thumbnail?id=1H-FhwpQ2zBOcdGEgOZ5EQEd9dHb4Ua75&sz=w1000'></img>

In [ ]:
new_wide_data = new_data.pivot(index=['country'],
                               columns = ['year'],
                               values =['fertility'])

# Flatten the MultiIndex in columns
new_wide_data.columns = new_wide_data.columns.get_level_values(1)

# Reset the index to make 'country' a regular column
new_wide_data = new_wide_data.reset_index()

new_wide_data

year,country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Germany,2.41,2.44,2.47,2.49,2.49,2.48,2.44,2.37,2.28,...,1.36,1.36,1.37,1.38,1.39,1.40,1.41,1.42,1.43,1.44
1,South Korea,6.16,5.99,5.79,5.57,5.36,5.16,4.99,4.85,4.73,...,1.20,1.21,1.23,1.25,1.27,1.29,1.30,1.32,1.34,1.36


#### Separating Variables

In [ ]:
combined_data = pd.read_csv('/content/life-expectancy-and-fertility-two-countries-example.csv')
combined_data

,country,1960_fertility,1960_life_expectancy,1961_fertility,1961_life_expectancy,1962_fertility,1962_life_expectancy,1963_fertility,1963_life_expectancy,1964_fertility,...,2011_fertility,2011_life_expectancy,2012_fertility,2012_life_expectancy,2013_fertility,2013_life_expectancy,2014_fertility,2014_life_expectancy,2015_fertility,2015_life_expectancy
0,Germany,2.41,69.26,2.44,69.85,2.47,70.01,2.49,70.10,2.49,...,1.40,80.5,1.41,80.6,1.42,80.7,1.43,80.7,1.44,80.8
1,South Korea,6.16,53.02,5.99,53.75,5.79,54.51,5.57,55.27,5.36,...,1.29,80.6,1.30,80.7,1.32,80.9,1.34,80.9,1.36,81.0


- We want to create a long table with four columns:
  - country
  - year
  - fertility
  - life_expectancy
- Steps:
  1. melt
  2. Create two new columns using str.split()
  3. pivot with country and value

In [ ]:
# 1. Melt
## We can skip value_vars, since we use the rest of columns
df_long = combined_data.melt(id_vars=['country'],
                  var_name='temp',
                  value_name='value')
df_long

,country,temp,value
0,Germany,1960_fertility,2.41
1,South Korea,1960_fertility,6.16
2,Germany,1960_life_expectancy,69.26
3,South Korea,1960_life_expectancy,53.02
4,Germany,1961_fertility,2.44
...,...,...,...
219,South Korea,2014_life_expectancy,80.90
220,Germany,2015_fertility,1.44
221,South Korea,2015_fertility,1.36
222,Germany,2015_life_expectancy,80.80


In [ ]:
# 2. Create two new columns based on str.split()
df_long[['year', 'variable']] = df_long['temp'].str.split('_', n=1, expand=True)
df_long

,country,temp,value,year,variable
0,Germany,1960_fertility,2.41,1960,fertility
1,South Korea,1960_fertility,6.16,1960,fertility
2,Germany,1960_life_expectancy,69.26,1960,life_expectancy
3,South Korea,1960_life_expectancy,53.02,1960,life_expectancy
4,Germany,1961_fertility,2.44,1961,fertility
...,...,...,...,...,...
219,South Korea,2014_life_expectancy,80.90,2014,life_expectancy
220,Germany,2015_fertility,1.44,2015,fertility
221,South Korea,2015_fertility,1.36,2015,fertility
222,Germany,2015_life_expectancy,80.80,2015,life_expectancy


In [ ]:
df_long = df_long.pivot(index=['country', 'year'],
                        columns='variable',
                        values='value').reset_index()
df_long

variable,country,year,fertility,life_expectancy
0,Germany,1960,2.41,69.26
1,Germany,1961,2.44,69.85
2,Germany,1962,2.47,70.01
3,Germany,1963,2.49,70.10
4,Germany,1964,2.49,70.66
...,...,...,...,...
107,South Korea,2011,1.29,80.60
108,South Korea,2012,1.30,80.70
109,South Korea,2013,1.32,80.90
110,South Korea,2014,1.34,80.90


## Joining Tables

- Sometimes, we need to combine a few data together.
- Pandas provide a function `merge` to join two data in various ways.

<img src= 'https://drive.google.com/thumbnail?id=1eCljI-KPaOhAAHow6Yj4qKfqu1mOhRIn&sz=w1000'></img>

#### joining example

In [ ]:
# load the data
murders = pd.read_csv('/content/murders.csv')
election2016 = pd.read_csv('/content/results_us_election_2016.csv')

In [ ]:
murders.head()

,state,abb,region,population,total
0,Alabama,AL,South,4779736,135
1,Alaska,AK,West,710231,19
2,Arizona,AZ,West,6392017,232
3,Arkansas,AR,South,2915918,93
4,California,CA,West,37253956,1257


In [ ]:
election2016.head()

,Unnamed: 0,state,electoral_votes,clinton,trump,others
0,1,California,55,61.7,31.6,6.7
1,2,Texas,38,43.2,52.2,4.5
2,3,Florida,29,47.8,49.0,3.2
3,4,New York,29,59.0,36.5,4.5
4,5,Illinois,20,55.8,38.8,5.4


In [ ]:
# Merge by state
first_merge =  pd.merge(murders, election2016, on='state', how='left')
first_merge

,state,abb,region,population,total,Unnamed: 0,electoral_votes,clinton,trump,others
0,Alabama,AL,South,4779736,135,22,9,34.4,62.1,3.6
1,Alaska,AK,West,710231,19,44,3,36.6,51.3,12.2
2,Arizona,AZ,West,6392017,232,14,11,45.1,48.7,6.2
3,Arkansas,AR,South,2915918,93,30,6,33.7,60.6,5.8
4,California,CA,West,37253956,1257,1,55,61.7,31.6,6.7
5,Colorado,CO,West,5029196,65,23,9,48.2,43.3,8.6
6,Connecticut,CT,Northeast,3574097,97,27,7,54.6,40.9,4.5
7,Delaware,DE,South,897934,38,45,3,53.4,41.9,4.7
8,District of Columbia,DC,South,601723,99,51,3,90.9,4.1,5.0
9,Florida,FL,South,19687653,669,3,29,47.8,49.0,3.2


#### Joining Example 2

- We will be looking at Tuberculosis in the United States.
- We can download this data from CDC https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down

In [ ]:
# More about this next week
#tbls = pd.read_html("https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm?s_cid=mm7112a1_w#T1_down")
#df = tbls[0] # First table on the website
#df.to_csv('cdc_tuberculosis.csv')

In [ ]:
tb_df = pd.read_csv("/content/cdc_tuberculosis.csv",)
tb_df

,Unnamed: 0,U.S. jurisdiction,No. of TB cases*,No. of TB cases*.1,No. of TB cases*.2,TB incidence†,TB incidence†.1,TB incidence†.2
0,NaN,U.S. jurisdiction,2019,2020,2021,2019.00,2020.00,2021.00
1,0.0,Total,8900,7173,7860,2.71,2.16,2.37
2,1.0,Alabama,87,72,92,1.77,1.43,1.83
3,2.0,Alaska,58,58,58,7.91,7.92,7.92
4,3.0,Arizona,183,136,129,2.51,1.89,1.77
5,4.0,Arkansas,64,59,69,2.12,1.96,2.28
6,5.0,California,2111,1706,1750,5.35,4.32,4.46
7,6.0,Colorado,66,52,58,1.15,0.90,1.00
8,7.0,Connecticut,67,54,54,1.88,1.50,1.50
9,8.0,Delaware,18,17,43,1.84,1.71,4.29


- The first row should be the column names/headers
- Always check your header

In [ ]:
tb_df = pd.read_csv("/content/cdc_tuberculosis.csv", header=1) # row index
tb_df = tb_df.drop(columns=tb_df.columns[0])
tb_df

,U.S. jurisdiction,2019,2020,2021,2019.1,2020.1,2021.1
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46
6,Colorado,66,52,58,1.15,0.90,1.00
7,Connecticut,67,54,54,1.88,1.50,1.50
8,Delaware,18,17,43,1.84,1.71,4.29
9,District of Columbia,24,19,19,3.39,2.75,2.84


- We need to differentiate "Number of TB cases" and "TB incidence"

In [ ]:
# Renaming columns
rename_dict = {'2019': 'TB cases 2019',
               '2020': 'TB cases 2020',
               '2021': 'TB cases 2021',
               '2019.1': 'TB incidence 2019',
               '2020.1': 'TB incidence 2020',
               '2021.1': 'TB incidence 2021'}
tb_df = tb_df.rename(columns=rename_dict)
tb_df

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
0,Total,8900,7173,7860,2.71,2.16,2.37
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46
6,Colorado,66,52,58,1.15,0.90,1.00
7,Connecticut,67,54,54,1.88,1.50,1.50
8,Delaware,18,17,43,1.84,1.71,4.29
9,District of Columbia,24,19,19,3.39,2.75,2.84


In [ ]:
# row = 0 is total. We don't need it.
tb_df = tb_df.drop(0) # drop index 0
tb_df.head()

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021
1,Alabama,87,72,92,1.77,1.43,1.83
2,Alaska,58,58,58,7.91,7.92,7.92
3,Arizona,183,136,129,2.51,1.89,1.77
4,Arkansas,64,59,69,2.12,1.96,2.28
5,California,2111,1706,1750,5.35,4.32,4.46


In [ ]:
tb_df.sum()

,0
U.S. jurisdiction,AlabamaAlaskaArizonaArkansasCaliforniaColorado...
TB cases 2019,8900
TB cases 2020,7173
TB cases 2021,7860
TB incidence 2019,107.23
TB incidence 2020,90.93
TB incidence 2021,100.57


- The total TB cases look right.
- The incidence numbers are not right.
- We need more information to calculate it correctly. **We need population information.**

#### Population information from Census data

- U.S. Census population estimates source ([2019](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html)), source ([2020-2021](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html)).

- There are a few new methods here:

  - `df.convert_dtypes()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.convert_dtypes.html)) conveniently converts all float dtypes into ints and is out of scope for the class.
  - `df.drop_na()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)) will be explained in more detail next time.

In [ ]:
census_2010s_df = pd.read_csv("/content/nst-est2019-01.csv", header=3, thousands=",")
census_2010s_df

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538.0,308758105.0,309321666.0,311556874.0,313830990.0,315993715.0,318301008.0,320635163.0,322941311.0,324985539.0,326687501.0,328239523.0
1,Northeast,55317240.0,55318443.0,55380134.0,55604223.0,55775216.0,55901806.0,56006011.0,56034684.0,56042330.0,56059240.0,56046620.0,55982803.0
2,Midwest,66927001.0,66929725.0,66974416.0,67157800.0,67336743.0,67560379.0,67745167.0,67860583.0,67987540.0,68126781.0,68236628.0,68329004.0
3,South,114555744.0,114563030.0,114866680.0,116006522.0,117241208.0,118364400.0,119624037.0,120997341.0,122351760.0,123542189.0,124569433.0,125580448.0
4,West,71945553.0,71946907.0,72100436.0,72788329.0,73477823.0,74167130.0,74925793.0,75742555.0,76559681.0,77257329.0,77834820.0,78347268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,Table 1. Annual Estimates of the Resident Popu...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Do some basic data cleaning

In [ ]:
census_2010s_df = (
    census_2010s_df
    .rename(columns={"Unnamed: 0": "Geographic Area"})
    .drop(columns=["Census", "Estimates Base"])
    .convert_dtypes() # "smart" converting of columns to int, use at your own risk
    .dropna()  # we'll introduce this very soon
)
census_2010s_df

,Geographic Area,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
1,Northeast,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803
2,Midwest,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004
3,South,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448
4,West,72100436,72788329,73477823,74167130,74925793,75742555,76559681,77257329,77834820,78347268
5,.Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
6,.Alaska,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
7,.Arizona,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
8,.Arkansas,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
9,.California,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223


Notice there is a '.' at the beginning of all the states. We need to remove that.



In [ ]:
census_2010s_df['Geographic Area'] = census_2010s_df['Geographic Area'].str.strip('.')
census_2010s_df

,Geographic Area,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
1,Northeast,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803
2,Midwest,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004
3,South,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448
4,West,72100436,72788329,73477823,74167130,74925793,75742555,76559681,77257329,77834820,78347268
5,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
6,Alaska,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
7,Arizona,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
8,Arkansas,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
9,California,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223


#### Loading the 2020s data

Census collected every 10 years so we need another file.

In [ ]:
# census 2020s data cleaning
census_2020s_df = pd.read_csv("/content/NST-EST2023-POP.csv", header=3, thousands=",")
census_2020s_df = (
    census_2020s_df
    .drop(columns=["Unnamed: 1"])
    .rename(columns={"Unnamed: 0": "Geographic Area"})
    .convert_dtypes()
    .dropna()
)
census_2020s_df['Geographic Area'] = census_2020s_df['Geographic Area'].str.strip('.')
census_2020s_df

,Geographic Area,2020,2021,2022,2023
0,United States,331526933,332048977,333271411,334914895
1,Northeast,57430477,57243423,57026847,56983517
2,Midwest,68969794,68850246,68783028,68909283
3,South,126465281,127353282,128702030,130125290
4,West,78661381,78602026,78759506,78896805
5,Alabama,5031864,5050380,5073903,5108468
6,Alaska,732964,734923,733276,733406
7,Arizona,7186683,7272487,7365684,7431344
8,Arkansas,3014348,3028443,3046404,3067732
9,California,39503200,39145060,39040616,38965193


#### Time to merge all three data
- tb data, census2010s, census2020s
- So that we have all population data from 2019-2021

In [ ]:
# merge TB dataframe with two US census dataframes
tb_census_df = (
    tb_df
    .merge(right=census_2010s_df,
           left_on="U.S. jurisdiction", right_on="Geographic Area")
    .merge(right=census_2020s_df,
           left_on="U.S. jurisdiction", right_on="Geographic Area")
)
tb_census_df.tail()

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,Geographic Area_x,2010,2011,...,2015,2016,2017,2018,2019,Geographic Area_y,2020,2021,2022,2023
46,Virginia,191,169,161,2.23,1.96,1.86,Virginia,8023699,8101155,...,8361808,8410106,8463587,8501286,8535519,Virginia,8637193,8657348,8679099,8715698
47,Washington,221,163,199,2.90,2.11,2.57,Washington,6742830,6826627,...,7163657,7294771,7423362,7523869,7614893,Washington,7724566,7741433,7784477,7812880
48,West Virginia,9,13,7,0.50,0.73,0.39,West Virginia,1854239,1856301,...,1842050,1831023,1817004,1804291,1792147,West Virginia,1791562,1785249,1774035,1770071
49,Wisconsin,51,35,66,0.88,0.59,1.12,Wisconsin,5690475,5705288,...,5760940,5772628,5790186,5807406,5822434,Wisconsin,5896700,5879978,5890543,5910955
50,Wyoming,1,0,3,0.17,0.00,0.52,Wyoming,564487,567299,...,585613,584215,578931,577601,578759,Wyoming,577664,579548,581629,584057


- We don't need all of these. Only 2019 - 2021

In [ ]:
# try merging again, but cleaner this time
tb_census_df = (
    tb_df
    .merge(right=census_2010s_df[["Geographic Area", "2019"]],
           left_on="U.S. jurisdiction", right_on="Geographic Area")
    .drop(columns="Geographic Area")
    .merge(right=census_2020s_df[["Geographic Area", "2020", "2021"]],
           left_on="U.S. jurisdiction", right_on="Geographic Area")
    .drop(columns="Geographic Area")
)
tb_census_df.tail()

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,2019,2020,2021
46,Virginia,191,169,161,2.23,1.96,1.86,8535519,8637193,8657348
47,Washington,221,163,199,2.90,2.11,2.57,7614893,7724566,7741433
48,West Virginia,9,13,7,0.50,0.73,0.39,1792147,1791562,1785249
49,Wisconsin,51,35,66,0.88,0.59,1.12,5822434,5896700,5879978
50,Wyoming,1,0,3,0.17,0.00,0.52,578759,577664,579548


#### Reproduce incidence

Let's recompute incidence to make sure we know where the original CDC numbers came from.

From the [CDC report](https://www.cdc.gov/mmwr/volumes/71/wr/mm7112a1.htm#T1_down): TB incidence is computed as “Cases per 100,000 persons using mid-year population estimates from the U.S. Census Bureau.”

If we define a group as 100,000 people, then we can compute the TB incidence for a given state population as

$\text{TB incidence} = \frac{\text{#TB cases in population}}{\text{population}} \times 100000$

In [ ]:
# Let's try this for 2019:
tb_census_df["recompute incidence 2019"] = (
    tb_census_df["TB cases 2019"]/tb_census_df["2019"] * 100_000
)
tb_census_df.head()

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,2019,2020,2021,recompute incidence 2019
0,Alabama,87,72,92,1.77,1.43,1.83,4903185,5031864,5050380,1.774357
1,Alaska,58,58,58,7.91,7.92,7.92,731545,732964,734923,7.928425
2,Arizona,183,136,129,2.51,1.89,1.77,7278717,7186683,7272487,2.514179
3,Arkansas,64,59,69,2.12,1.96,2.28,3017804,3014348,3028443,2.120747
4,California,2111,1706,1750,5.35,4.32,4.46,39512223,39503200,39145060,5.342651


Let's use a for-loop and Python format strings to compute TB incidence for all years.

In [ ]:
# recompute incidence for all years
for year in [2019, 2020, 2021]:
    tb_census_df[f"recompute incidence {year}"] = (
        round(tb_census_df[f"TB cases {year}"]/tb_census_df[f"{year}"]*100_000,2)
    )
tb_census_df

,U.S. jurisdiction,TB cases 2019,TB cases 2020,TB cases 2021,TB incidence 2019,TB incidence 2020,TB incidence 2021,2019,2020,2021,recompute incidence 2019,recompute incidence 2020,recompute incidence 2021
0,Alabama,87,72,92,1.77,1.43,1.83,4903185,5031864,5050380,1.77,1.43,1.82
1,Alaska,58,58,58,7.91,7.92,7.92,731545,732964,734923,7.93,7.91,7.89
2,Arizona,183,136,129,2.51,1.89,1.77,7278717,7186683,7272487,2.51,1.89,1.77
3,Arkansas,64,59,69,2.12,1.96,2.28,3017804,3014348,3028443,2.12,1.96,2.28
4,California,2111,1706,1750,5.35,4.32,4.46,39512223,39503200,39145060,5.34,4.32,4.47
5,Colorado,66,52,58,1.15,0.90,1.00,5758736,5785219,5811596,1.15,0.9,1.0
6,Connecticut,67,54,54,1.88,1.50,1.50,3565287,3577586,3603691,1.88,1.51,1.5
7,Delaware,18,17,43,1.84,1.71,4.29,973764,991862,1004881,1.85,1.71,4.28
8,District of Columbia,24,19,19,3.39,2.75,2.84,705749,670839,669037,3.4,2.83,2.84
9,Florida,558,412,499,2.60,1.91,2.29,21477737,21591299,21830708,2.6,1.91,2.29


### Some words about Date and Time
- Time stamp is another common data
- Need to be careful about time and date formatting!!
- US is MM/DD/YY. TH is DD/MM/YY. JP is YY/MM/DD.

# ---- Break -----

## Dealing with Text

## Why work with text? Two common goals

1. **Canonicalization**:  Convert data that has more than one possible presentation into a standard form.
  - Ex: Join tables with mismatched labels

<img src= 'https://drive.google.com/thumbnail?id=1qpza0rUz-MuzC9hQ4cafSXB8ujXzgGwe&sz=w1000' width = 750></img>

## Why work with text? Two common goals

1. **Canonicalization**:  Convert data that has more than one possible presentation into a standard form.
  - Ex: Join tables with mismatched labels

2. **Extract information** into a new feature (column)
  - Ex: Extract data and times from log files

<img src= 'https://drive.google.com/thumbnail?id=1JDfMlTbwPPGcEzadWHLjaIxi1XAo7sr8&sz=w1000' width = 600></img>

### Python String Methods

- Transformation: `s.lower()`, `s.upper()`
- Split: `s.split(...)`
- membership: `'ab' in s`
- replacement: `s.replace(substring, newstring)`
- substring: `s[1:4]`
- length: `len(s)`

**Problem**: Work on one string and not a list of string. Slow when you need to for loop in a large table.

### Pandas String Methods
- Pandas provides a method of **vectorizing** text operations: the `.str` operator
- **Syntax**: `Series.str.<string_operation>()`
- Apply the function `<string_operation>` to every string contained in the Series



In [ ]:
# Load data for this section
states = pd.read_csv("/content/county_and_state.csv")
populations = pd.read_csv("/content/county_and_population.csv")

# display allows us to view a DataFrame without returning it as an object
display(states)
display(populations)

,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


In [ ]:
#Ex1 str.lower()
populations['County'].str.lower()

,County
0,dewitt
1,lac qui parle
2,lewis & clark
3,st. john the baptist


In [ ]:
#Ex2 str.replace()
populations['County'].str.replace('&','and')

,County
0,DeWitt
1,Lac Qui Parle
2,Lewis and Clark
3,St. John the Baptist


### `.str` Methods


<img src= 'https://drive.google.com/thumbnail?id=12hXiKY3DgFqNjgtcq4T1SDqCO9CjiBRe&sz=w1000' width = 750></img>

#### Merging these two datasets
- we need to **canonicalize** the "County" string data to apply a common formatting.

In [ ]:
def canonicalize_county(county_series):
    return (county_series.str.lower()               # lowercase
            .str.replace(' ', '')                   # remove space
            .str.replace('&', 'and')                # replace &
            .str.replace('.', '')                   # remove dot
            .str.replace('county', '')              # remove "county"
            .str.replace('parish', '')              # remove "parish"
            )

display(canonicalize_county(states["County"]))
display(canonicalize_county(populations["County"]))

,County
0,dewitt
1,lacquiparle
2,lewisandclark
3,stjohnthebaptist


,County
0,dewitt
1,lacquiparle
2,lewisandclark
3,stjohnthebaptist


In [ ]:
states["Canonical County"] = canonicalize_county(states["County"])
populations["Canonical County"] = canonicalize_county(populations["County"])
display(states)
display(populations)

,County,State,Canonical County
0,De Witt County,IL,dewitt
1,Lac qui Parle County,MN,lacquiparle
2,Lewis and Clark County,MT,lewisandclark
3,St John the Baptist Parish,LS,stjohnthebaptist


,County,Population,Canonical County
0,DeWitt,16798,dewitt
1,Lac Qui Parle,8067,lacquiparle
2,Lewis & Clark,55716,lewisandclark
3,St. John the Baptist,43044,stjohnthebaptist


Now, the merge works as expected!

In [ ]:
states.merge(populations, on="Canonical County")

,County_x,State,Canonical County,County_y,Population
0,De Witt County,IL,dewitt,DeWitt,16798
1,Lac qui Parle County,MN,lacquiparle,Lac Qui Parle,8067
2,Lewis and Clark County,MT,lewisandclark,Lewis & Clark,55716
3,St John the Baptist Parish,LS,stjohnthebaptist,St. John the Baptist,43044


### Extracting Log Data

<img src= 'https://drive.google.com/thumbnail?id=1JDfMlTbwPPGcEzadWHLjaIxi1XAo7sr8&sz=w1000' width = 600></img>

In [ ]:
# Loading the log file
log_fname = '/content/log.txt'
with open(log_fname, 'r') as f:
    log_lines = f.readlines()
log_lines

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n']

- We want to extract the day, month, year, hour, minutes, seconds, and timezone.
- Let's start with the first line.

In [ ]:
first = log_lines[0]
first

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

- Get the data inside the square brackes by **splitting** string at the square brackets

In [ ]:
pertinent = (
    first.split("[")[1] # remove everything before the first [
    .split(']')[0] # Remove everything after the second square ]
) # find the text enclosed in square brackets
pertinent

'26/Jan/2014:10:47:58 -0800'

- Next, split up date by `/`

In [ ]:
day, month,rest  = pertinent.split('/')       # split up the date/month/year

print("Day:   ", day)
print("Month: ", month)
print("Rest:  ", rest)

Day:    26
Month:  Jan
Rest:   2014:10:47:58 -0800


- Next, split up the hour:minute:second

In [ ]:
year, hour, minute, rest = rest.split(':')    # split up the hour:minute:second

print("Year:   ", year)
print("Hour:   ", hour)
print("Minute: ", minute)
print("Rest:   ", rest)

Year:    2014
Hour:    10
Minute:  47
Rest:    58 -0800


- Lastly, split the timezone after the blank space

In [ ]:
seconds, time_zone = rest.split(' ')          # split the timezone after the blank space
day, month, year, hour, minute, seconds, time_zone

('26', 'Jan', '2014', '10', '47', '58', '-0800')

#### Now, we will do it with Pandas `.str`

In [ ]:
logs = pd.read_csv("/content/log.txt", header=None)[0] #get Series! (Why?)

print("Original input!")
display(logs)

Original input!


,0
0,169.237.46.168 - - [26/Jan/2014:10:47:58 -0800...
1,"193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] ""..."
2,"169.237.46.240 - """" [3/Feb/2006:10:18:37 -0800..."


In [ ]:
# Split [] and then /
df = (
    logs.str.split("[")
        .str[1]
        .str.split("]")
        .str[0]
        .str.split("/", expand=True)
        .rename(columns={0: "Day", 1: "Month", 2: "Rest"})
)
df

,Day,Month,Rest
0,26,Jan,2014:10:47:58 -0800
1,2,Feb,2005:17:23:6 -0800
2,3,Feb,2006:10:18:37 -0800


In [ ]:
# split :
df = (
    df.join(df["Rest"].str.split(":", expand=True))
        .drop(columns=["Rest"])
        .rename(columns={0: "Year", 1: "Hour", 2: "Minute", 3: "Rest"})
)
df

,Day,Month,Year,Hour,Minute,Rest
0,26,Jan,2014,10,47,58 -0800
1,2,Feb,2005,17,23,6 -0800
2,3,Feb,2006,10,18,37 -0800


In [ ]:
# split space
df = (
    df.join(df["Rest"].str.split(" ", expand=True))
        .drop(columns=["Rest"])
        .rename(columns = {0: "Seconds", 1: "Timezone"})
)

# done
df

,Day,Month,Year,Hour,Minute,Seconds,Timezone
0,26,Jan,2014,10,47,58,-0800
1,2,Feb,2005,17,23,6,-0800
2,3,Feb,2006,10,18,37,-0800


## Regular Expressions
- A **reg**ular **ex**pression (“regex”) is a sequence of characters that specifies a search pattern.
- It is an embedded domain specific programming language (**a tiny programming language**)

### String Extraction with Regex

- Python `re.findall` returns a list of **all extracted matches**:

In [ ]:
#Example
import re

text = "My phone number is 017-456-6789 bro, or actually maybe it’s 321-425-6789.";

pattern = r"[0-9]{3}-[0-9]{3}-[0-9]{4}"

re.findall(pattern, text)

['017-456-6789', '321-425-6789']

### Goals for Regex

The goal of today is **NOT** to memorize the language of regular expressions!
Instead:
- Understand what regex is capable of.
- Parse and create regex, **with a reference table**.




### Learn from Practice
- Use vocabulary (metacharacter, escape character, groups, etc.) to **describe regex** metacharacters.
- **Differentiate** between `(), [], {}`
- Design your own **character classes** with `\d, \w, \s, […-…], ^,` etc.
- Use Python and pandas regex methods.

### Resources for Practicing Regex
- [regex101.com](regex101.com): provides a visually appealing and easy to use platform for experimenting with regular expressions.
  - **Important**: choose the Python “flavor” in the left sidebar

<img src= 'https://drive.google.com/thumbnail?id=1G9qRRnzueiMogfWDCI7zOIOE9ROsyr6l&sz=w1000' width = 800></img>

### Regex Basics
- There are four basic operations in regex

##### Concentenation - "look for consecutive characters"

- `AABAAB` matches `AABAAB`

#### | - “or”
- `AA|BAAB` matches `AA` or `BAAB`

#### * - “zero or more”
- `AB*A` matches `AA, ABA, ABBA, …`

#### ( ) - “consider a group”
- `(AB)*A` matches `A, ABA, ABABA, …`
- `A(A|B)AAB` matches `AAAAB` or `ABAAB`


- `*`, `( )`, and `|` are called **metacharacters** - they represent an operation, rather than a literal text character


### Basic Regex

<img src= 'https://drive.google.com/thumbnail?id=1P15x87nJG3iOtbvvXamsHg5S-SDTrQN4&sz=w1000' width = 800></img>

#### Ex1 (Slido)

Which regex patterns will match the string "ABBA"? Select all that apply.


#### Ex2 (Slido)

Which pattern matches moon, moooon, etc?

Your expression should match any *even* number of “o”s except zero (i.e. don't match mn, mooon).


#### Try it yourself

- https://regex101.com/r/8tkQ23/1
- https://regex101.com/r/kJpHeZ/1



### 6 More Regex Operations

#### `.` - “look for any character”

- `.U.U.U.` matches `CUMULUS`, `JUGULUM`



#### `+` - “one or more”

- `AB+` matches `AB, ABB, ABBB, …`



#### `?` - “zero or one” ("optional")
- `AB?` matches `A, AB`


#### `{x}` - “repeat exactly x times”
- `AB{2}` matches `ABB`


#### `{x, y}` - “repeat between x and y times”
- `AB{0,2}` matches `A, AB, ABB`

#### `[ ]` - “define a character class”
- `[A-Za-z]` matches `A, a, B, b…`
- `[A-Z]` - any uppercase letter between `A` and `Z`
- `[0-9]` (or `\d`) - any digit between `0` and `9`
- `[A-Za-z0-9]` (or `\w`) - any letter, any digit

Use ^ to negate a class = match any character other than what follows
- `[^A-Z]` - anything that is *not* an uppercase letter between A and Z

### Regex Expand

<img src= 'https://drive.google.com/thumbnail?id=1Q_5IC03bfxXwlg1He5wIC9NBG4Dd1ZVu&sz=w1000' width = 800></img>

#### Ex3 (Slido)

Choose a cleaner solution to the example from before! Which pattern matches moon, moooon, etc?

#### Ex4 (Slido)

What pattern matches any lowercase alphabetic string that has a repeated vowel (noon, peel, festoon, loop, oodles, etc)?


### Regex is greedy

- Regex is **greedy** - it will look for the longest possible match in a string

- Example: `<div>.*</div>`
- Demo: https://regex101.com/r/HATiTH/1

- In English:
  - “Look for the exact string **\<div>**"
  - then, “look for any character 0 or more times”
  - then, “look for the exact string **\</div>**"

- “This is a **\<div>**example\</div> of greediness \<div>in**\</div>** regular expressions.”

- We can fix this by making the pattern **non-greedy**:
  - `<div>.*?</div>`
  - `?` makes the multipliers non-greedy.


### More Regex Operations

#### `\` - 'read the next character literally'
- `a\+b` mataches `a+b`

#### `^` - “match the beginning of a string”
- `^abc` does not match `123 abc`


#### `$` - “match the end of a string”
- `abc$` does not match `abc 123`



### Extra Regex Expand

<img src= 'https://drive.google.com/thumbnail?id=1iVhxSRjF1T8xXFO5W598jLVMJt0TSAzD&sz=w1000' width = 800></img>

### Regex Functions

#### Before We Begin: Raw Strings in Python
- When specifying a pattern, we strongly suggest using raw strings.
- A raw string is created by prepending `r` to the string delimiters
  - `(r"...", r'...', r"""...""", r'''...''')`
- Don't need to escape special characters with `\`
  - e.g. `"\\\\section"` = `r"\\section"`


### Extraction: `re.findall(pattern, text)`
- `re.findall(pattern, text)`: Return a list of all matches to `pattern`.
- `ser.str.findall(pattern)`: Return a Series of lists.

In [ ]:
# ex: findall
text = "My phone number is 017-456-6789 bro, or actually maybe it’s 321-425-6789.";

pattern = r"[0-9]{3}-[0-9]{3}-[0-9]{4}"

re.findall(pattern, text)

['017-456-6789', '321-425-6789']

In [ ]:
# ex: .str.findall
df_phones = pd.DataFrame(
    ['987-653-4321',
     'forty',
     '123-453-6789 bro or 321-451-6789',
     '999-999-9999'],
    columns=['Phone'])
df_phones

,Phone
0,987-653-4321
1,forty
2,123-453-6789 bro or 321-451-6789
3,999-999-9999


In [ ]:
df_phones['Phone'].str.findall(pattern)

,Phone
0,[987-653-4321]
1,[]
2,"[123-453-6789, 321-451-6789]"
3,[999-999-9999]


### Extraction with Capture Groups
- Earlier we used parentheses to specify the **order of operations**.
- Parenthesis can have another meaning:
  - When using certain regex functions, ( ) specifies a capture group.
  - Extract only the portion of the regex pattern inside the capture group


In [ ]:
text = """I will meet you at 08:30:00 pm tomorrow"""
pattern = ".*(\d\d):(\d\d):(\d\d).*"
matches = re.findall(pattern, text)
matches

[('08', '30', '00')]

### Extraction with Capture Groups
- `ser.str.extract(pattern)`: Returns a DataFrame of each capture group's **first** match in the string
- `ser.str.extractall(pattern)`: Returns a multi-indexed DataFrame of **all** matches for each capture group


In [ ]:
# Will extract the first match of all groups
pattern_group_mult = r"([0-9]{3})-([0-9]{3})-([0-9]{4})" # 3 groups
df_phones['Phone'].str.extract(pattern_group_mult)

,0,1,2
0,987,653,4321
1,NaN,NaN,NaN
2,123,453,6789
3,999,999,9999


In [ ]:
# -> DataFrame, one row per match
df_phones['Phone'].str.extractall(pattern_group_mult)

0    1     2
  match                
0 0      987  653  4321
2 0      123  453  6789
  1      321  451  6789
3 0      999  999  9999

#### Substitution
- `re.sub(pattern, repl, text)`: Returns text with all instances of `pattern` replaced by `repl`.


In [ ]:
# In regular Python, canonicalize with re.sub (standing for "substitute"):
text = '<div><td valign="top">Moo</td></div>'
pattern = r"<[^>]+>"
re.sub(pattern, '', text)

'Moo'

How it works:
- `pattern` matches HTML tags
- Then, sub/replace HTML tags with `repl=''` (i.e., empty string)


- `ser.str.replace(pattern, repl, regex=True )`: Returns Series with all instances of `pattern` in Series `ser` replaced by `repl`.

In [ ]:
# example dataframe of strings
df_html = pd.DataFrame(['<div><td valign="top">Moo</td></div>',
                   '<a href="http://ds100.org">Link</a>',
                   '<b>Bold text</b>'], columns=['Html'])
df_html

,Html
0,"<div><td valign=""top"">Moo</td></div>"
1,"<a href=""http://ds100.org"">Link</a>"
2,<b>Bold text</b>


In [ ]:
# Series -> Series
df_html["Html"].str.replace(pattern, '', regex=True).to_frame()

,Html
0,Moo
1,Link
2,Bold text


### Revisiting Text Log Processing using Regex


In [ ]:
line = log_lines[0]
display(line)

# (numbers)/(Words)/(Numbers):(numbers):(numbers):(numbers):(numbes) (any)
pattern = r'\[(\d+)\/(\w+)\/(\d+):(\d+):(\d+):(\d+) (.+)\]'

day, month, year, hour, minute, second, time_zone = re.findall(pattern, line)[0] # get first match
day, month, year, hour, minute, second, time_zone

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

('26', 'Jan', '2014', '10', '47', '58', '-0800')

In [ ]:
df = pd.DataFrame(log_lines, columns=['Log'])
df

,Log
0,169.237.46.168 - - [26/Jan/2014:10:47:58 -0800...
1,"193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] ""..."
2,"169.237.46.240 - """" [3/Feb/2006:10:18:37 -0800..."


In [ ]:
pattern = r'\[(\d+)\/(\w+)\/(\d+):(\d+):(\d+):(\d+) (.+)\]'
df['Log'].str.extractall(pattern)

,,0,1,2,3,4,5,6
,match,,,,,,,
0,0,26,Jan,2014,10,47,58,-0800
1,0,2,Feb,2005,17,23,6,-0800
2,0,3,Feb,2006,10,18,37,-0800


### Pandas Summary


<img src= 'https://drive.google.com/thumbnail?id=1Vi4EoCKG5IoUAurQ0ihkOWmt4Stag-PM&sz=w1000' width = 800></img>

### Limitations of Regular Expressions
- Need to know the syntax well
- Hard to read and debug
- Reality: Just ask AI...